<a href="https://colab.research.google.com/github/shivakumarsanugula/Kaggle-NLP/blob/main/introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#check for GPU

!nvidia-smi

Tue Dec  5 09:59:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b7c68e2a-e407-3b17-8dba-a386f18de60c)


In [3]:
# Get helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-12-05 09:59:16--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-12-05 09:59:16 (70.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
## Get a text dataset

!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#unzip data
unzip_data("nlp_getting_started.zip")

--2023-12-05 09:59:20--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.126.207, 74.125.132.207, 74.125.201.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.126.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-12-05 09:59:20 (151 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
## Visualizing a text dataset
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [7]:
#shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# what does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# How many total samples?
len(train_df),len(test_df)

(7613, 3263)

In [11]:
#Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df) -5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target:{target}","(real diaster)" if target > 0 else "(not real diaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target:1 (real diaster)
Text:
@bluebirddenver #FETTILOOTCH IS #SLANGLUCCI OPPRESSIONS GREATEST DANGER COMING SOON THE ALBUM 
https://t.co/moLL5vd8yD

---

Target:1 (real diaster)
Text:
iembot_hfo : At 10:00 AM 2 NNW Hana [Maui Co HI] COUNTY OFFICIAL reports COASTAL FLOOD #Û_ http://t.co/Gg0dZSvBZ7) http://t.co/kBe91aRCdw

---

Target:0 (not real diaster)
Text:
@AnnmarieRonan @niamhosullivanx I can't watch tat show its like a horror movie to me I get flashbacks an everything #traumatised

---

Target:0 (not real diaster)
Text:
Brooklyn locksmith: domesticate emergency mechanic services circa the clock movement!: gba http://t.co/1Q6ccFfzV6

---

Target:1 (real diaster)
Text:
A look at state actions a year after Ferguson's upheaval http://t.co/vXUFtVT9AU

---



In [12]:
#Splitting data into training and validation sets

from sklearn.model_selection import train_test_split

# use train_test_split to split training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,random_state=42)

In [13]:
# check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
# check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [15]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
# converting text data to numbers using tokenisation and embeddings

# Text vectorization(tokenization)
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [17]:
# use the defalut TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=10000,# how many words in the vocabulary (automatically add <00v>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,# create group of n-words
                                    output_mode="int", #how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be
                                    pad_to_max_tokens=True)

In [18]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [19]:
len(train_sentences[0].split())

7

In [20]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [21]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary it is hyper-parameter
max_length = 15 # max length our sequences will be (ex:- how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [22]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [23]:
# Create a sample sentence and tokenize it

sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [24]:
# choose a random sentence from the training data and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVectorized version: ")
text_vectorizer([random_sentence])

Original text:
 Hi @FionaGilbert_ sorry for the delay. Slightly inundated with applications at the mo but we have yours and will get back to you asap!        

Vectorized version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1663,    1,  875,   10,    2, 3942, 1976,  507,   14, 4137,   17,
           2, 2382,   30,   46]])>

In [25]:
# Get the unique words in the vocublary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words:{bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [26]:
#[UNK] unkowun token

# Creating an Embedding using an Embedding Layer
'''
To make our embedding, we're going to use TensorFlow's embedding layer :
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about for our embedding layer:

* input_dim = the size of the vocabulary
* output_dim = the size the output embedded vector, for example, a value would mean each token gets represented by a vector 100 long
* input_length = length of the sequences being passed to the embedding layer

'''

In [27]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # output shape works well which is divisible by 8
                             input_length=max_length # how long is each input
                             )

In [28]:
embedding

In [29]:
# Get a random setence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
       \n\nEmbedded version: ")

# Embed the random sentence (turn into  dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Japan Marks 70th Anniversary of Hiroshima Atomic Bombing http://t.co/93vqkdFgnr       

Embedded version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02591661,  0.02672372, -0.00436749, ..., -0.02962583,
          0.03360548,  0.01452212],
        [ 0.04362119,  0.02705136, -0.03795337, ..., -0.00337227,
         -0.04408132,  0.03898371],
        [ 0.00342949, -0.01202447, -0.01324711, ..., -0.00450931,
         -0.02420774, -0.04222798],
        ...,
        [ 0.02760089, -0.04559756, -0.0087082 , ..., -0.00676087,
         -0.03009833, -0.04292152],
        [ 0.02760089, -0.04559756, -0.0087082 , ..., -0.00676087,
         -0.03009833, -0.04292152],
        [ 0.02760089, -0.04559756, -0.0087082 , ..., -0.00676087,
         -0.03009833, -0.04292152]]], dtype=float32)>

In [30]:
# check out a single token's embedding
sample_embed[0][0],sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.02591661,  0.02672372, -0.00436749,  0.03078712, -0.03347689,
        -0.01976109, -0.01014348,  0.02169735, -0.00674726,  0.0422059 ,
        -0.00747752, -0.00807701,  0.01666081, -0.01621304,  0.03280807,
        -0.01722036, -0.00431643, -0.03096552, -0.02643833,  0.04231874,
        -0.02203974, -0.03845879, -0.02030351,  0.03153571, -0.02298763,
         0.0476804 , -0.03790376,  0.02675905,  0.00071407, -0.01281785,
        -0.02664378, -0.01756456,  0.03372501,  0.02899027, -0.00635394,
         0.02367401,  0.02510146, -0.04648011, -0.02728183,  0.02594787,
         0.00474924, -0.0491225 ,  0.02274607, -0.0369745 , -0.0457264 ,
         0.03892199,  0.00897572,  0.02520677, -0.00525542, -0.02626264,
        -0.00048246,  0.02223426,  0.01585499, -0.03366752, -0.04941155,
        -0.03748815,  0.00697989, -0.01662376,  0.04614044, -0.01660264,
        -0.00031431, -0.01890388,  0.03360541, -0.03497847,  0.01708633,
  

# Modelling a text dataset (running a series of experiments)

Now we've got way to turn our text sequence into numbers, it's time to start building a series of modelling experiments.

* Model 0 : Naive Bayes (baseline), this is from sklearn ML map: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
* Model 1 : Feed-forward neural network (dense model)
* Model 2 : LSTM model (RNN)
* Model 3 : GRU model (RNN)
* Model 4 : Bidirectional- LSTM model (RNN)
* Model 5 : 1D Convolutional Neural Network (CNN)
* Model 6 : TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7 :  same as model 6 with 10% of training data

How are we going to apporach all of these?

Use the standard steps in modelling with tensorflow:
* Create a model
* Build a model
* Fit a model
* Evaluate our model


#Model 0 : Getting a baseline

In [31]:
# Model 0 : Getting a Baseline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tf-idf
                    ("clf", MultinomialNB()) # model the text
])

#Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [32]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of : {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of : 79.27%


In [33]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [34]:
## Creating an evalution function for our model experiments
# For deepview of metrices: https://scikit-learn.org/stable/modules/model_evaluation.html

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculaute_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary calssification model.
  """
  #Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  #Calculate model precision, recall, and f1-score using "weighted average"
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy" : model_accuracy,
                   "precision" : model_precision,
                   "recall" : model_recall,
                   "f1": model_f1}
  return model_results

In [35]:
# Get baseline results
baseline_results = calculaute_results(y_true=val_labels,
                                      y_pred=baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#Model 1 : Feed-forward neural network (dense model)

In [36]:
# A simple dense model

# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [37]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x) # Create the output layer, want binary outputs so use sigmoid
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")  # construct the model


In [38]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [39]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20231205-095926
Epoch 1/5
215/215 [==============================] - 27s 91ms/step - loss: 0.6111 - accuracy: 0.6916 - val_loss: 0.5380 - val_accuracy: 0.7546
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4409 - accuracy: 0.8177 - val_loss: 0.4691 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3468 - accuracy: 0.8612 - val_loss: 0.4580 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2840 - accuracy: 0.8913 - val_loss: 0.4700 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2377 - accuracy: 0.9115 - val_loss: 0.4770 - val_accuracy: 0.7861


In [40]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 6ms/step - loss: 0.4770 - accuracy: 0.7861


[0.4769909977912903, 0.7860892415046692]

In [41]:
# Make some predictions and evalaute those

model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 3ms/step


(762, 1)

In [42]:
# look at a single prediction
model_1_pred_probs[0]

array([0.3899171], dtype=float32)

In [43]:
# look at a first 10 prediction
model_1_pred_probs[:10]

array([[0.3899171 ],
       [0.8203959 ],
       [0.9971455 ],
       [0.14513029],
       [0.11534972],
       [0.9483334 ],
       [0.89280224],
       [0.99332774],
       [0.9649343 ],
       [0.3365645 ]], dtype=float32)

In [44]:
# Convert model prediction probabilites to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [45]:
# Calculate our model_1 results
model_1_results = calculaute_results(y_true=val_labels,
                                     y_pred=model_1_preds)
model_1_results

{'accuracy': 78.60892388451444,
 'precision': 0.7895616761419161,
 'recall': 0.7860892388451444,
 'f1': 0.7835741000287678}

In [46]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [47]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [48]:
## Visualizing learned embedding

# Get the vocabulary from the text vectorization layer

words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [49]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [50]:
# Get the weight matrix of embedding layer
# (these are the numerical representations of each token in our trainingdata, which have been learned for ~5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape) # same size as vocab size and embedding_dim (output_dim of our embedding layer)

(10000, 128)


Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TensorFlow has a handy tool called projector : https://projector.tensorflow.org/

And Tensorflow also has an incredible guide on word embeding themselves : https://www.tensorflow.org/text/guide/word_embeddings

In [51]:
embed_weights

array([[ 0.0153187 , -0.03909537,  0.00313334, ...,  0.00652206,
        -0.02088682, -0.05641435],
       [ 0.000688  ,  0.03363376,  0.02473783, ...,  0.01193349,
         0.01605979,  0.03480062],
       [-0.04813533, -0.02361766,  0.06087554, ...,  0.02348975,
         0.04658139,  0.0007195 ],
       ...,
       [ 0.03242662,  0.00833423, -0.01005337, ...,  0.03980961,
        -0.00660457, -0.00056978],
       [-0.0099221 ,  0.07170857, -0.00282814, ...,  0.02291023,
         0.04709361, -0.02661534],
       [-0.020154  ,  0.05269286,  0.08345308, ...,  0.04327415,
         0.06152309, -0.09159228]], dtype=float32)

In [52]:
# Creating embedding files (we got this from Tensorflow's word embeddings documentation)
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [53]:
# Download files from colab to upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Network (RNN)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

LSTM : https://colah.github.io/posts/2015-08-Understanding-LSTMs/



## Model 2 : LSTM
LSTM = long short term memory (one of the most opular LSTM cells)

Our structure of an RNN typically looks like this:

Input(text) -> Tokenize -> Embedding -> Layers (RNNs/Dense) -> Output(label probability)

In [54]:
# Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#print(x.shape)
#x = layers.LSTM(64, return_sequences=True)(x) # when you're stacking RNN cells together, you need to set return_sequences=True
#print(x.shape)
x = layers.LSTM(64)(x)
#print(x.shape)
#x = layers.Dense(64, activation='relu')(x)
#print(x.shape)
outputs = layers.Dense(1,activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs,outputs,name="model_2_LSTM")

In [55]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [56]:
# compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [57]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20231205-100012
Epoch 1/5
215/215 [==============================] - 11s 38ms/step - loss: 0.2197 - accuracy: 0.9222 - val_loss: 0.6148 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1549 - accuracy: 0.9418 - val_loss: 0.6423 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1267 - accuracy: 0.9520 - val_loss: 0.7048 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1013 - accuracy: 0.9615 - val_loss: 0.8411 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0795 - accuracy: 0.9676 - val_loss: 1.1108 - val_accuracy: 0.7756


In [58]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[6.5612420e-02],
       [6.2559056e-01],
       [9.9933428e-01],
       [5.3209201e-03],
       [1.1010247e-04],
       [9.9527800e-01],
       [5.8844417e-01],
       [9.9970418e-01],
       [9.9945670e-01],
       [5.2395433e-01]], dtype=float32)

In [59]:
# convert model_2 into labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [60]:
# Calculate model 2 results
model_2_results = calculaute_results(y_true=val_labels,
                                     y_pred=model_2_preds)
model_2_results

{'accuracy': 77.55905511811024,
 'precision': 0.7769617885542762,
 'recall': 0.7755905511811023,
 'f1': 0.7737412623092624}

In [61]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 3 : GRU (Gated recurrent unit)

The GRU cell has similar features to an LSTM cell but has less parameters

In [62]:
# Build an RNN using the GRU cell

from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
#print(x.shape)
#x = layers.GRU(64, return_sequences=True)(x)
#print(x.shape)
#x = layers.LSTM(42,return_sequences=True)(x)
#print(x.shape)
#x = layers.GRU(99)(x)
#print(x.shape)
#x = layers.Dense(64, activation="relu")(x)
#x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")


In [63]:
# Get summary of model
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

In [64]:
# Comiple the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [65]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20231205-100033
Epoch 1/5
215/215 [==============================] - 12s 44ms/step - loss: 0.1539 - accuracy: 0.9404 - val_loss: 0.7177 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0832 - accuracy: 0.9686 - val_loss: 0.7786 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0700 - accuracy: 0.9726 - val_loss: 0.8856 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0601 - accuracy: 0.9742 - val_loss: 0.9852 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0512 - accuracy: 0.9781 - val_loss: 1.3702 - val_accuracy: 0.7756


In [66]:
# Make some predictions with our GRU model
model_3_pred_prob = model_3.predict(val_sentences)
model_3_pred_prob[:10]

24/24 [==============================] - 0s 3ms/step


array([[2.5134705e-04],
       [5.7410079e-01],
       [9.9961686e-01],
       [4.4882223e-03],
       [3.9648683e-05],
       [9.9917334e-01],
       [4.4470313e-01],
       [9.9990451e-01],
       [9.9980170e-01],
       [6.0698926e-01]], dtype=float32)

In [67]:
# convert model 3 pred into labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_prob))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [68]:
# Calculate model 3 results
model_3_results = calculaute_results(y_true=val_labels,
                                     y_pred=model_3_preds)
model_3_results

{'accuracy': 77.55905511811024,
 'precision': 0.7810725814480683,
 'recall': 0.7755905511811023,
 'f1': 0.7720383798735819}

## Model 4: Bidirectional LSTM RNN


In [69]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
#x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [70]:
# Get a summary
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1378945 (5.26 MB)
Trainable par

In [71]:
# Comiple the model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [72]:
# Fit the model
model_4_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20231205-100056
Epoch 1/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0490 - accuracy: 0.9783 - val_loss: 1.1426 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0430 - accuracy: 0.9810 - val_loss: 1.4050 - val_accuracy: 0.7651
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0505 - accuracy: 0.9772 - val_loss: 1.0734 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0470 - accuracy: 0.9794 - val_loss: 1.2548 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0380 - accuracy: 0.9822 - val_loss: 1.3863 - val_accuracy: 0.7717


In [73]:
# Make some predictions with our GRU model
model_4_pred_prob = model_4.predict(val_sentences)
model_4_pred_prob[:10]

24/24 [==============================] - 1s 3ms/step


array([[0.50539464],
       [0.4941625 ],
       [0.52204895],
       [0.5033992 ],
       [0.48855507],
       [0.5095614 ],
       [0.50453645],
       [0.5390153 ],
       [0.525339  ],
       [0.5111907 ]], dtype=float32)

In [74]:
# convert model 4 pred into labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_prob))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 0., 1., 1., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [75]:
# Calculate model 3 results
model_4_results = calculaute_results(y_true=val_labels,
                                     y_pred=model_4_preds)
model_4_results

{'accuracy': 62.20472440944882,
 'precision': 0.6699631225935105,
 'recall': 0.6220472440944882,
 'f1': 0.6091772369282342}

# Model 5 : Conv1D

In [77]:
# Test out our embedding layer, conv1d layer and max pooling

from  tensorflow.keras import layers
embedding_test = embedding(text_vectorizer(['this is a test sentences']))
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation='relu',
                        padding='valid')
conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # equivalent to get the most important feature or get the feature with the highest value
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))